# Zero shot learning - BART-LARGE MNLI

## Import Libraries

In [1]:
import pandas as pd
import numpy as np

from transformers import pipeline
from sklearn.model_selection import KFold

from sklearn.metrics import f1_score, accuracy_score

from tqdm.notebook import tqdm

from collections import Counter
from scipy import stats

In [3]:
model = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0", device="cuda") 

In [13]:
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p5.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha3_p5.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha3_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha3_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha3_p2neg.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha3_p2neg.csv")

### Load experiments for alpha 4
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p5.csv")
df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha4_p5.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha4_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha4_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha4_p2neg.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha4_p2neg.csv")

### Load experiments for alpha 5
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p5.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha5_p5.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha5_p3.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha5_p2plus.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/percept_dataset_alpha5_p2neg.csv")
# df = pd.read_csv("/home/neemias/PerceptSent-LLM-approach/data/gpt4-openai-classify/percept_dataset_alpha5_p2neg.csv")

df.head()

,id,text,sentiment
0,1BhIkrf3HWatQFG7KxJK7MZ1oWbUczaar,The image depicts a long line of people waitin...,1
1,15LVGeXllPyPe1jicnPTPl6kbBYJ5lZh8,The image depicts an expansive attic space cha...,4
2,11_yZVpq48fA-jchrZO_sTtUKQejXd-te,"The image shows a black car, likely a luxury s...",1
3,1gnbr9y1VBnFSqyqxLU-W5WrsN6DejtdG,The image features a scenic landscape dominate...,4
4,1UMwl8aaZiEf0O6iswA4UXU_ejf1DqSlt,"The image features several trees with thick, r...",3


## Sentiment Analyzer

In [14]:
import warnings
warnings.filterwarnings("ignore")
df_metrics = pd.DataFrame([])
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_idx, val_idx) in enumerate(kfold.split(df)):
    train_df = pd.DataFrame({"text": df["text"].iloc[train_idx].to_list(), 
                                     "sentiment": df["sentiment"].iloc[train_idx].to_list()})
    val_df = pd.DataFrame({"text": df["text"].iloc[val_idx].to_list(), 
                                "sentiment": df["sentiment"].iloc[val_idx].to_list()})
    

    text = val_df["text"].to_list()
    target = val_df["sentiment"].to_list()

    sentiments = np.unique(target)

    if (len(sentiments) == 3):
        sent_dic = {
            "Negative": 0,
            "Neutral": 1,
            "Positive": 2,
        }
        sents = ["Positive", "Neutral", "Negative"]
    elif (len(sentiments) == 2):
        # # P2+
        # sent_dic = {
        #     "Negative": 1,
        #     "Positive": 0,
        # }
        # sents = ["Negative", "Positive"]
        # P2 -
        sent_dic = {
            "Negative": 0,
            "Positive": 1,
        }
        sents = ["Negative", "Positive"]
    else:
        sent_dic = {
            "Positive": 4,
            "SlightlyPositive": 3,
            "Neutral": 2,
            "SlightlyNegative": 1,
            "Negative": 0
        }
        sents = ["Positive", "SlightlyPositive", "Neutral", "SlightlyNegative", "Negative"]

    pred = []
    for t in text:
        result = model(t, sents)
        pred.append(sent_dic[result["labels"][0]])
    accuracy_val = accuracy_score(target, pred)
    f1_val = f1_score(target, pred, average="weighted")    
    df_metrics = pd.concat([df_metrics, pd.DataFrame({"accuracy": [accuracy_val], "f1_score": [f1_val]
                                                         })], axis=0)
df_metrics.head()

,accuracy,f1_score
0,0.666667,0.622833
0,0.673401,0.617862
0,0.619529,0.564434
0,0.662162,0.595066
0,0.655405,0.601039


In [15]:
f1_scores = df_metrics["f1_score"].to_list()
mean_f1 = np.mean(f1_scores)


# define the confidence level
confidence_level = 0.95
degrees_freedon = len(f1_scores)-1

confidence_interval = stats.t.interval(
    confidence_level, 
    degrees_freedon, 
    loc=mean_f1, 
    scale=stats.sem(f1_scores)
)

print(f"Max F1-score: {max(f1_scores)}")
print(f"Average F1-score: {mean_f1}")
print(f"Confidence interval 95%: {confidence_interval}")
print(f"Inteval: {abs(confidence_interval[0]-mean_f1)} - Interval: {abs(confidence_interval[1]-mean_f1)}")
confidence_interval = stats.t.interval(
    confidence_level, 
    degrees_freedon, 
    loc=max(f1_scores), 
    scale=stats.sem(f1_scores)
)

print(f"Inteval: {abs(confidence_interval[0]-max(f1_scores))} - Interval: {abs(confidence_interval[1]-max(f1_scores))}")

Max F1-score: 0.6228329618244451
Average F1-score: 0.600246735838668
Confidence interval 95%: (0.571591187479813, 0.6289022841975231)
Inteval: 0.028655548358855087 - Interval: 0.028655548358855087
Inteval: 0.028655548358855087 - Interval: 0.028655548358855087
